In [ ]:
import isodisreg 
from isodisreg import idr
import properscoring as ps
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
import torch.nn.functional as F
import torchvision.models
import netCDF4 as nc4
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
from joblib import Parallel, delayed
import ipywidgets as widgets
from ipywidgets import interact
from scipy.ndimage import zoom
from datetime import datetime
import utils as ut
import dependency as dep
import models64 as models
device = torch.device("cuda" if torch.cuda.is_available() else "CPU")
GPM = '/projectnb/labci/NAS/Data/GPM/Version6/Ghana/pr_20000602S_20210930E/'
TIG = '/projectnb/labci/NAS/Project/ShortTermRainfallForecast_Ghana/version1/code/Inference/Data/'

In [ ]:
def plot_data(time_step, cbar_label='Data Value', save=None):
    num_models = len(models)
    fig, axes = plt.subplots(1, num_models, figsize=(14 * num_models / 2, 8), constrained_layout=False, facecolor='white')
    if num_models == 1:
        axes = [axes]
    fig.subplots_adjust(wspace=0.05, hspace=0.05)
    global_min = min(np.min(data[time_step, :, :]) for data in models.values())
    global_max = max(np.max(data[time_step, :, :]) for data in models.values())

    for i, (ax, (title, data)) in enumerate(zip(axes, models.items())):
        ax.set_facecolor('white')
        m = Basemap(projection='cyl', llcrnrlat=latitude.min(), urcrnrlat=latitude.max(),
                    llcrnrlon=longitude.min(), urcrnrlon=longitude.max(), resolution='l', ax=ax)

        m.drawcoastlines(linewidth=1.5, color='darkred')
        m.drawcountries(linewidth=1.5, color='darkred')
        m.drawparallels(np.arange(int(latitude.min()), int(latitude.max()) + 1, 1), labels=[1, 0, 0, 0], fontsize=20)
        m.drawmeridians(np.arange(int(longitude.min()), int(longitude.max()) + 1, 1), labels=[0, 0, 0, 1], fontsize=20)

        x, y = m(longitude, latitude)
        cs = m.pcolormesh(x, y, data[time_step, :, :], cmap='Blues', shading='auto', vmin=global_min, vmax=global_max)
        if '_' in title:
            base_name, suffix = title.rsplit('_', 1)  # Split title into base name and suffix
            ax.set_title(f"{base_name}$_{{{suffix}}}$", fontsize=24)
        else:
            ax.set_title(f"{title}", fontsize=24)


        # Add colorbar only to the rightmost plot
        if i == num_models - 1:
            cbar = m.colorbar(cs, location='right', pad="5%")
            cbar.set_label(cbar_label, fontsize=24)
            #cbar.set_ticks([-1, 0, 1])
            #cbar.ax.set_yticklabels(['-1', '0', '1'], fontsize=20)
            cbar.ax.tick_params(labelsize=20)
         # Save only the first image
        if save:
            plt.savefig(save, bbox_inches='tight', dpi=600)
            print(f"Saved first image as {save}")
    plt.show()
with open('/projectnb/labci/NAS/Project/ShortTermRainfallForecast_Ghana/version1/saved_data/lonlat_GPM_ERA.pkl', 'rb') as f:
    GPMLong, GPMLat, _,_ = pickle.load(f)
new_lat = np.linspace(GPMLat.max(), GPMLat.min(), 64)
new_long = np.linspace(GPMLong.min(), GPMLong.max(), 64)
latitude, longitude = np.array(new_lat), np.array(new_long)
longitude, latitude = np.meshgrid(longitude, latitude)  # Create 2D grid

In [ ]:
# get the training data for CRPS
#GPM original dimension=(54, 73) and ERA original dimension=(30, 23). Change the dimension of one dataset
with open('/projectnb/labci/NAS/Project/ShortTermRainfallForecast_Ghana/version1/saved_data/date_all.pkl', 'rb') as f:
    date = pickle.load(f)
with open('/projectnb/labci/NAS/Project/ShortTermRainfallForecast_Ghana/version1/saved_data/model3/ERA5_64_64.pkl', 'rb') as f:
    ERA = pickle.load(f)
with open('/projectnb/labci/NAS/Project/ShortTermRainfallForecast_Ghana/version1/saved_data/model3/GPM_64_64.pkl', 'rb') as f:
    GPM = pickle.load(f)
    #GPM = np.clip(GPM, 0, None)
print(GPM.shape, ERA.shape, np.min(GPM), np.max(GPM), date.shape, date)
#Normalization
Ndata = []
for i in range(len(ERA)):
    if(i<59):
        Ndata.append(ut.znorm(ERA[i]))
    else:
        print('No normalization for as they are seasonal and time variables',i)
        Ndata.append([ERA[i],0,0])
ENorm = [item[0] for item in Ndata]
Emean = [item[1] for item in Ndata] 
Estd = [item[2] for item in Ndata]
GNorm, Gmean, Gstd  = ut.norm(GPM)
def Preparedata(data,dataL,s,d,var, date):
    X,Y,Z = [],[],[]
    for i in range(0,len(data[0])-max(s,d)):
        T, L = [], []
        for j in var:
            x = data[j][i:i+s]
            T.append(x)
        X.append(T),Y.append(dataL[i+s:i+s+d]),Z.append(int(date[i+s:i+s+d]))#Y.append(data[0][i+s])
    return np.array(X), np.array(Y), np.array(Z)

sequence, days, variable = 1, 1, [num for num in range(61) if num < 2 or num > 5]
X,Y,Z = Preparedata(ENorm, GNorm, sequence, days, variable, date) #time sequence=3, can be 1 to 10; can send Ndata Ndata for normalize input and output
if(len(variable)>1):
    X = X.reshape(X.shape[0], X.shape[1]*X.shape[2], X.shape[3], X.shape[4])
    #Y = np.expand_dims(Y,axis=1) # 0 represents the precipition data as output
X = X.squeeze()
###################Important#############################################
num_images = len(X)
split_ratio = [0.98, 0.01, 0.01]
num_train = int(num_images * split_ratio[0])
num_val = int(num_images * split_ratio[1])
num_test = num_images - num_train - num_val
if os.path.exists('/projectnb/labci/NAS/Project/ShortTermRainfallForecast_Ghana/version1/saved_data/permutation_indices.npy'):
    print("Permutation file exists and loading it...")
    permutation_indices = np.load('/projectnb/labci/NAS/Project/ShortTermRainfallForecast_Ghana/version1/saved_data/permutation_indices.npy')
else:
    print("Permutation file does not exist and preparing it for future reference")
    permutation_indices = np.random.permutation(num_images)
    #np.save('/projectnb/labci/Indrajit/Rainfall/data/ERA5/Version3/data/permutation_indices.npy', permutation_indices)
X,Y,Z = X[permutation_indices],Y[permutation_indices],Z[permutation_indices]

trainX, valX, testX = np.split(X, [num_train, num_train + num_val])
trainY, valY, testY = np.split(Y, [num_train, num_train + num_val])
trainZ, valZ, testZ = np.split(Z, [num_train, num_train + num_val])
GPM_train = ut.CustomDataset(trainX, trainY)
GPM_val = ut.CustomDataset(valX, valY)
GPM_test = ut.CustomDataset(testX, testY)
print(len(GPM_train), len(GPM_val), len(GPM_test))
batch_size = 1
train_loader = DataLoader(GPM_train, batch_size=batch_size, shuffle=True)

In [ ]:
rainy_pixels = GPM>=0
percent_rainy_pixels = np.sum(rainy_pixels) / GPM.size * 100
# Percentage of rainy days (any pixel > threshold)
rainy_days = np.any(rainy_pixels, axis=(1, 2))
percent_rainy_days = np.sum(rainy_days) / GPM.shape[0] * 100

print(f"Percentage of rainy pixels (all days): {percent_rainy_pixels:.2f}%")
print(f"Percentage of rainy days (any pixel): {percent_rainy_days:.2f}%")

In [ ]:
# First do the evaluation for TIGG dataset
GPM = '<GPM dataset path>'
TIG = '<ECMWF Ensemble path>'
GroundTruth, TIGG_Pturb, TIGG_Control, Date_Position, TargetDate = dep.Evaluation(GPM, TIG, GPM_zoom=True, GPM_desired_shape = (64, 64),
                                                                              TIG_zoom=True, TIG_desired_shape = (64, 64))

TIGG_Pred = np.mean(np.stack((TIGG_Pturb, TIGG_Control), axis=0), axis=0)
print('TIGG prediction shape', TIGG_Pred.shape)
Ind_TIG = np.array(dep.Indv_TIGG(TIG, TIG_zoom=True, TIG_desired_shape = (64, 64)))
Ind_TIG_ens = np.concatenate((Ind_TIG, np.expand_dims(TIGG_Control, axis=1)), axis=1)# Combine the Ensemble of TIGG (50+1)
print('TIGG shape for ensemble and CRPS', Ind_TIG_ens.shape)
Random_Prediction, Ind_Random, Random_GT = dep.RandomEvaluation(GPM, TargetDate, GPM_zoom=True, GPM_desired_shape = (64, 64))

In [ ]:
## For model. First load the model and the test_loader
model, test_loader, Gmean, Gstd, norm = dep.load_model()
model1, test_loader1, _, _, _ = dep.load_model()
model2, test_loader2, _, _, _ = dep.load_model()
model3, test_loader3, _, _, _ = dep.load_model()

In [ ]:
# For model. First select the date that is available in TIGG and then create the loader and evaluate
subset_inputs, subset_targets = dep.extract_ordered_subset_from_loader(test_loader, Date_Position)
subset_dataset = TensorDataset(subset_inputs, subset_targets)
subset_inputs1, subset_targets1 = dep.extract_ordered_subset_from_loader(test_loader1, Date_Position)
subset_dataset1 = TensorDataset(subset_inputs1, subset_targets1)
subset_inputs2, subset_targets2 = dep.extract_ordered_subset_from_loader(test_loader2, Date_Position)
subset_dataset2 = TensorDataset(subset_inputs2, subset_targets2)
subset_inputs3, subset_targets3 = dep.extract_ordered_subset_from_loader(test_loader3, Date_Position)
subset_dataset3 = TensorDataset(subset_inputs3, subset_targets3)
# Create a DataLoader with the subset dataset based on TIGG data
subset_loader = DataLoader(subset_dataset, batch_size=1, shuffle=False)
subset_loader1 = DataLoader(subset_dataset1, batch_size=1, shuffle=False)
subset_loader2 = DataLoader(subset_dataset2, batch_size=1, shuffle=False)
subset_loader3 = DataLoader(subset_dataset3, batch_size=1, shuffle=False)
def ModelPredict(subset_loader, model):
    Data = list() # For prediction
    Target = list() # For Ground Truth
    C = 0
    for input, target in subset_loader:
            input, target = input.to(device),target.to(device)
            output = model(input)
            T,P = target[0,0].cpu().detach().numpy(), output[0,0].cpu().detach().numpy()
            if norm == 'znorm':
                T,P = ut.iznorm(T, Gmean, Gstd), ut.iznorm(P, Gmean, Gstd)
            if norm == 'norm':
                T,P = ut.inorm(T, Gmean, Gstd), ut.inorm(P, Gmean, Gstd)
            Data.append(P)
            Target.append(T)
    return np.array(Data), np.array(Target)
ModelPrediction, Model_GT = ModelPredict(subset_loader, model)
ModelPrediction1, _ = ModelPredict(subset_loader1, model1)
ModelPrediction2, _ = ModelPredict(subset_loader2, model2)
ModelPrediction3, _ = ModelPredict(subset_loader3, model3)
#-------------------------------------------------------------------#
TrainPred, Train_GT = ModelPredict(train_loader, model)
TrainPred1, _ = ModelPredict(train_loader, model1)
TrainPred2, _ = ModelPredict(train_loader, model2)
TrainPred3, _ = ModelPredict(train_loader, model3)

In [ ]:
# Hybrid model: Linera regression: Y - UNET = W1*(NWP-UNET) + W2
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
Months = np.arange(0, 55)
UNET_flat = ModelPrediction2[Months].reshape(-1)
TIGG_flat = TIGG_Pred[Months].reshape(-1)
GPM_flat = Model_GT[Months].reshape(-1)
#-------------------------------Compute difference -------------------------------------#
X_diff = TIGG_flat - UNET_flat 
Y_diff = GPM_flat - UNET_flat
#------------------------------------------------------------------------------------------
# Train the linear regression model
regressor = LinearRegression(fit_intercept=False)
regressor.fit(X_diff.reshape(-1, 1), Y_diff)
# Get the learned weights and intercept
W1, W2 = regressor.coef_[0], regressor.intercept_
print(f"W1 (weight): {W1}, W2 (intercept): {W2}")
#------------------------------------------------------------------------------------------
ensemble_avg = np.zeros_like(ModelPrediction2)
for i in range(ensemble_avg.shape[0]):  # Loop over images
    for j in range(ensemble_avg.shape[1]):  # Loop over rows (pixels)
        for k in range(ensemble_avg.shape[2]):  # Loop over columns (pixels)
            dif = TIGG_Pred[i, j, k] - ModelPrediction2[i, j, k]
            val = regressor.predict(dif.reshape(-1,1)) #Y - UNET
            ensemble_avg[i, j, k] = val + ModelPrediction2[i, j, k]  #Y = pred + UNET 
print("Ensemble Matrix Shape:", ensemble_avg.shape)
#------------------------------------------------------------------------------------------

In [ ]:
def visual(Bar=False, save_path=None, **images):
    """Plot images in one row with a single color bar and optionally save the plot."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    # Find the common color range
    vmin = min(image.min() for image in images.values())
    vmax = max(image.max() for image in images.values())
    # Generate random locations once
    first_image = list(images.values())[0]
    x_locs = np.random.randint(0, first_image.shape[1], 3)
    y_locs = np.random.randint(0, first_image.shape[0], 3)
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        if '_' in name:
            base_name, suffix = name.rsplit('_', 1)  # Split name and suffix
            plt.title(f"{base_name}$_{{{suffix}}}$")
        else:
            plt.title(name)
        img = plt.imshow(image, cmap='viridis', vmin=vmin, vmax=vmax)  # Use common color range
    if Bar:
        cbar = plt.colorbar(img, ax=plt.gca(), orientation='vertical', fraction=0.02, pad=0.04)
        cbar.ax.tick_params(labelsize=15)
    
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=1000)
    plt.show()
I = [49, 14, 34]
for r in range(0,len(I)):
    i = I[r]
    PATH = './Result/SamplePredImage'+str(i)+'.png'
    visual(Bar=True, save_path=None, GT=Model_GT[i], CLIM=Random_Prediction[i], NWP=TIGG_Pred[i],
           UNET_18=ModelPrediction2[i], UNET_12=ModelPrediction3[i], HYB = ensemble_avg[i])

In [ ]:
def visual_grid(image_dict_rows, save_path=None):
    nrows = len(image_dict_rows)
    ncols = max(len(row) for row in image_dict_rows)

    # Find global vmin/vmax
    vmin = min(image.min() for row in image_dict_rows for image in row.values())
    vmax = max(image.max() for row in image_dict_rows for image in row.values())

    fig, axes = plt.subplots(nrows, ncols, figsize=(3 * ncols, 3 * nrows),
                             constrained_layout=False, squeeze=False)

    for i, row in enumerate(image_dict_rows):
        for j, (name, image) in enumerate(row.items()):
            ax = axes[i, j]
            im = ax.imshow(image, cmap='Blues', vmin=vmin, vmax=vmax)
            ax.set_xticks([])
            ax.set_yticks([])

            # Add titles only on top row
            if i == 0:
                if '_' in name:
                    base_name, suffix = name.rsplit('_', 1)
                    ax.set_title(f"{base_name}$_{{{suffix}}}$", fontsize=12)
                else:
                    ax.set_title(name, fontsize=15)
        # Hide unused axes in this row
        for j in range(len(row), ncols):
            axes[i, j].axis('off')

    # Colorbar at bottom
    cbar_ax = fig.add_axes([0.2, 0.05, 0.6, 0.02])  # [left, bottom, width, height]
    cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
    cbar.ax.tick_params(labelsize=14)

    # Reduce spacing between images
    plt.subplots_adjust(wspace=0.03, hspace=0.03, bottom=0.09)

    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=900)

    plt.show()

visual_grid([
    {'GT': Model_GT[49], 'CLIM': Random_Prediction[49], 'NWP': TIGG_Pred[49], 'UNET_18': ModelPrediction2[49], 'UNET_12': ModelPrediction3[49], 'HYB': ensemble_avg[49]},
    {'GT': Model_GT[14], 'CLIM': Random_Prediction[14], 'NWP': TIGG_Pred[14], 'UNET_18': ModelPrediction2[14], 'UNET_12': ModelPrediction3[14], 'HYB': ensemble_avg[14]},
    {'GT': Model_GT[34], 'CLIM': Random_Prediction[34], 'NWP': TIGG_Pred[34], 'UNET_18': ModelPrediction2[34], 'UNET_12': ModelPrediction3[34], 'HYB': ensemble_avg[34]}],
            save_path = None)#'./Result/SamplePredImage1.png'

In [ ]:
#The Evaluation. MAE (GroundTruth, Prediction)
def mae_and_std(Target, Data):
    mae_values = np.mean(np.abs(Data - Target), axis=0) #MAE per location across all samples
    print(mae_values.shape, Target.shape)
    avg_mae = np.mean(mae_values) #Mean of MAE
    avg_std = np.std(mae_values) #STD of MAE
    return avg_mae, avg_std
    
TIGG_MAE, random_MAE = mae_and_std(Model_GT, TIGG_Pred), mae_and_std(Random_GT, Random_Prediction)
model_MAE, model_MAE1, model_MAE2, model_MAE3 = mae_and_std(Model_GT, ModelPrediction), mae_and_std(Model_GT, ModelPrediction1), mae_and_std(Model_GT, ModelPrediction2), mae_and_std(Model_GT, ModelPrediction3)
ensemble_MAE = mae_and_std(Model_GT, ensemble_avg)
# Print header
print("\n{:<25} {:<15} {:<15}".format("Model", "MAE", "STD"))
print("=" * 50)
# Print results in a structured format
print("{:<25} {:<15.6f} {:<15.6f}".format("1. CLIM", random_MAE[0], random_MAE[1]))
print("{:<25} {:<15.6f} {:<15.6f}".format("2. TIGG", TIGG_MAE[0], TIGG_MAE[1]))
print("{:<25} {:<15.6f} {:<15.6f}".format("3. UNET_18", model_MAE2[0], model_MAE2[1]))
print("{:<25} {:<15.6f} {:<15.6f}".format("4. UNET_12", model_MAE3[0], model_MAE3[1]))
print("{:<25} {:<15.6f} {:<15.6f}".format("5. HYB", ensemble_MAE[0], ensemble_MAE[1]))
print("=" * 50)


In [ ]:
# Save the error matrix:
M = TIGG_Pred - Model_GT
#np.save("./Result/NWP_Samplewise_Error.npy", M)
M = Random_Prediction - Random_GT
#np.save("./Result/CLIM_Samplewise_Error.npy", M)
M = ModelPrediction2 - Model_GT
#np.save("./Result/UNET18_Samplewise_Error.npy", M)
M = ModelPrediction3 - Model_GT
#np.save("./Result/UNET12_Samplewise_Error.npy", M)
M = ensemble_avg - Model_GT
#np.save("./Result/HYB_Samplewise_Error.npy", M)

In [ ]:
def preparehyb_data(Ind_TIG_ens, ModelPrediction2, regressor):
    dif = Ind_TIG_ens - ModelPrediction2[:, np.newaxis, :, :]
    dif_flat = dif.reshape(-1, 1)
    val_flat = regressor.predict(dif_flat)
    val = val_flat.reshape(Ind_TIG_ens.shape)
    ensemble_avg = val + ModelPrediction2[:, np.newaxis, :, :]
    return ensemble_avg

In [ ]:
# CRPS
def compute_crps(x_values, y):
    n = len(x_values)
    x_values = sorted(x_values)  # Ensure the data is sorted
    k = next((i for i in range(n-1) if x_values[i] <= y <= x_values[i+1]), n-1) # Find k such that x_k <= y <= x_k+1
    crps = 0
    #----------------- Sum for j < k --------------------------------------#
    for j in range(k):
        crps += (x_values[j+1] - x_values[j]) * ( (j+1)/n )**2
    #------------------ Contribution from the interval [x_k, x_k+1]------------------#
    if k < n - 1:  # Ensure x_k+1 exists
        crps += ((k+1)/n)**2 * (y - x_values[k]) + ((k+1)/n - 1)**2 * (x_values[k+1] - y)    
    #----------------- Sum for j > k --------------------------------------#
    for j in range(k+1, n-1):
        crps += (x_values[j+1] - x_values[j]) * ( (j+1)/n - 1 )**2
    return crps
def manual_crps(Ind_Random, Random_GT):
    crps_clim = []
    for i in range(len(Ind_Random)):
        map = np.zeros((64, 64))
        if np.mean(Random_GT[i])>0.0:
            for j in range(Random_GT.shape[1]):
                for k in range(Random_GT.shape[2]):
                    x_values, y = np.array(Ind_Random[i])[:,j,k], Random_GT[i,j,k]
                    map[j,k] = compute_crps(x_values, y)
            crps_clim.append(map)
            #print(f'The CRPS for the image is: {np.mean(map)} and std dev. is {np.std(map)}')
    crps_clim = np.mean(np.array(crps_clim), axis=0)
    #print(np.mean(crps_clim))
    return crps_clim

def compute_crps_per_pixel(i, j, train_preds, train_target, val_preds, val_target):
    """ Compute CRPS for a single pixel (i, j). """
    idr_per_grid = idr(y=train_target[:, i, j], X=pd.DataFrame(train_preds[:, i, j]))
    val_dist_pred = idr_per_grid.predict(pd.DataFrame(val_preds[:, i, j]))
    return np.mean(val_dist_pred.crps(val_target[:, i, j]))

def calculate_crps_idr_parallel(val_preds, val_target, train_preds, train_target):
    grid_size = 64  # Assuming a 64x64 grid
    crps_map = np.zeros((grid_size, grid_size))
    results = Parallel(n_jobs=-1)(delayed(compute_crps_per_pixel)(i, j, train_preds, train_target, val_preds, val_target)
                                  for i in range(grid_size) for j in range(grid_size))

    crps_map = np.array(results).reshape(grid_size, grid_size)
    return crps_map


crps_clim = manual_crps(Ind_Random, Random_GT)
crps_tigg = manual_crps(Ind_TIG_ens, GroundTruth)
crps_model2 = calculate_crps_idr_parallel(ModelPrediction2, Model_GT, TrainPred2, Train_GT) #EASYUQ
crps_model3 = calculate_crps_idr_parallel(ModelPrediction3, Model_GT, TrainPred3, Train_GT)
crps_hyb = manual_crps(preparehyb_data(Ind_TIG_ens, ModelPrediction2, regressor), Model_GT)

print("\n{:<25} {:<15}".format("Model", "CRPS Mean | STD"))  # Print header
print("=" * 42)
print("{:<25} {:<7.6f} | {:<7.6f}".format("1. CLIM", np.mean(crps_clim), np.std(crps_clim)))
print("{:<25} {:<7.6f} | {:<7.6f}".format("2. TIGG", np.mean(crps_tigg), np.std(crps_tigg)))
print("{:<25} {:<7.6f} | {:<7.6f}".format("3. UNET_18", np.mean(crps_model2), np.std(crps_model2)))
print("{:<25} {:<7.6f} | {:<7.6f}".format("4. UNET_12", np.mean(crps_model3), np.std(crps_model3)))
print("{:<25} {:<7.6f} | {:<7.6f}".format("5. HYB", np.mean(crps_hyb), np.std(crps_hyb)))
print("=" * 42)

In [ ]:
def Skill_score (ref, binary = True, **models):
    n = len(models)
    skill_scores = []
    for name, pred in models.items():
        skill = (ref - pred)/ref
        if binary: skill = np.where(skill < 0, -1, np.where(skill > 0, 1, 0))
        skill_scores.append(skill)
    return skill_scores

skills = Skill_score(crps_clim, binary=True, TIG=crps_tigg, UNET_18=crps_model2, UNET_12=crps_model3, HYB=crps_hyb)
#visual(Bar=True, save_path=None, CLIM_crps = crps_clim, TIGG=crps_tigg, Model2=crps_model2, Model3=crps_model3)
#visual(Bar=True, save_path=None, TIGG = skills[0], UNET_18 = skills[1], UNET_12 = skills[2])
models = {"CLIM": np.expand_dims(crps_clim,axis=0),"NWP": np.expand_dims(crps_tigg, axis=0),"UNET_18": np.expand_dims(crps_model2, axis=0),
          "UNET_12": np.expand_dims(crps_model3, axis=0),"HYB": np.expand_dims(crps_hyb, axis=0)}
interact(lambda time_step: plot_data(time_step, cbar_label='CRPS score', save='./Result/CRPS.png'),
         time_step=widgets.IntSlider(min=0, max=Model_GT.shape[0]-1, step=1, value=0));

models = {"NWP": np.expand_dims(skills[0], axis=0), "UNET_18": np.expand_dims(skills[1], axis=0),
          "UNET_12": np.expand_dims(skills[2], axis=0), "HYB": np.expand_dims(skills[3], axis=0)}
interact(lambda time_step: plot_data(time_step, cbar_label='CRPS Skill', save='./Result/CRPS_skill.png'),
         time_step=widgets.IntSlider(min=0, max=Model_GT.shape[0]-1, step=1, value=0));

In [ ]:
#bin_edges = [-1, -0.1, 0, 0.1, 1] + list(np.linspace(1, 5, num=48))
bin_edges = [-1, -0.1, 0, 0.1, 1]
#bin_edges = [-1, 0, 1]
# Target and Data should be numpy arrays of shape (N, W, H)
hist1, bin_edges1 = dep.compute_histogram(Random_GT, Random_Prediction, bin_edges, Print=False, Hist=False)
hist2, bin_edges2 = dep.compute_histogram(GroundTruth, TIGG_Pred, bin_edges, Print=False, Hist=False)
hist3, bin_edges3 = dep.compute_histogram(Model_GT, ModelPrediction2, bin_edges, Print=False, Hist=False)
hist4, bin_edges4 = dep.compute_histogram(Model_GT, ModelPrediction3, bin_edges, Print=False, Hist=False)
hist5, bin_edges5 = dep.compute_histogram(Model_GT, ensemble_avg, bin_edges, Print=False, Hist=False)

# Create column headers dynamically based on bin ranges
header = ["Model"] + [f"Range {i} ({bin_edges[i]} to {bin_edges[i+1]})" for i in range(len(bin_edges)-1)]

# Print header
print("\n" + "{:<20}".format(header[0]) + " ".join(["{:<15}".format(h) for h in header[1:]]))
print("=" * (20 + 15 * (len(bin_edges)-1)))
# Print histogram values for each model
print("{:<20} {}".format("1. CLIM", " ".join("{:<15}".format(h) for h in hist1)))
print("{:<20} {}".format("2. TIGG", " ".join("{:<15}".format(h) for h in hist2)))
print("{:<20} {}".format("3. UNET_18", " ".join("{:<15}".format(h) for h in hist3)))
print("{:<20} {}".format("4. UNET_12", " ".join("{:<15}".format(h) for h in hist4)))
print("{:<20} {}".format("5. HYB", " ".join("{:<15}".format(h) for h in hist5)))
print("=" * (20 + 15 * (len(bin_edges)-1)))

In [ ]:
#The Evaluation: TS/CSI, FPR, POD (GroundTruth, Prediction, threshold)
#1/100 (0.01) of an inch of rain – The first measurable amount of rainfall reported by The National Weather Service
def score(ground_truth, forecast, threshold):
    TP, FP, FN, TN = [], [], [], []
    for sample in range(len(ground_truth)):
        gt_mask = ground_truth[sample] > threshold
        forecast_mask = forecast[sample] > threshold
        TP.append(np.logical_and(forecast_mask, gt_mask))
        FP.append(np.logical_and(forecast_mask, np.logical_not(gt_mask)))
        FN.append(np.logical_and(np.logical_not(forecast_mask), gt_mask))
        TN.append(np.logical_and(np.logical_not(forecast_mask), np.logical_not(gt_mask)))
    TP, FP, FN, TN = map(lambda x: np.array(x).astype(int), [TP, FP, FN, TN])
    TP_sum, FP_sum, FN_sum, TN_sum = np.sum(TP, axis=0), np.sum(FP, axis=0), np.sum(FN, axis=0), np.sum(TN, axis=0)
    precision = np.divide(TP_sum, TP_sum + FP_sum, out=np.zeros_like(TP_sum, dtype=float), where=(TP_sum + FP_sum)!=0)
    CSI = np.divide(TP_sum, TP_sum + FP_sum + FN_sum, out=np.zeros_like(TP_sum, dtype=float), where=(TP_sum + FP_sum + FN_sum)!=0)
    recall = np.divide(TP_sum, TP_sum + FN_sum, out=np.zeros_like(TP_sum, dtype=float), where=(TP_sum + FN_sum)!=0)
    accuracy = np.divide(TP_sum + TN_sum, TP_sum + FP_sum + FN_sum + TN_sum, out=np.zeros_like(TP_sum, dtype=float), 
                         where=(TP_sum + FP_sum + FN_sum + TN_sum)!=0)
    return precision, recall, CSI, accuracy

th = [0.5, 10]
for t in th:
    print('\n-------------------------The Results based on the threshold=', t, '---------------------------')
    thresholds = [t]
    print("\n{:<20} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format("Model", "CSI", "Precision", "Recall", "F1", "P_loc", "R_loc", "F1_loc"))
    print("=" * 95)
    
    Csi, Precision, Recall, F1, ACC = dep.score_calculate(Random_GT, Random_Prediction, thresholds)
    Random_P, Random_R, Random_C, Random_ACC = score(Random_GT,Random_Prediction, t)
    print("{:<20} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f}"
          .format("1. CLIM", Csi, Precision, Recall, F1, np.mean(Random_P), np.mean(Random_R), np.mean(Random_C)))
    print(ACC)
    
    Csi, Precision, Recall, F1, ACC = dep.score_calculate(GroundTruth, TIGG_Pred, thresholds)
    TIGG_P, TIGG_R, TIGG_C, TIGG_ACC = score(GroundTruth,TIGG_Pred, t)
    print("{:<20} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f}"
          .format("2. TIGG",Csi, Precision, Recall, F1, np.mean(TIGG_P), np.mean(TIGG_R), np.mean(TIGG_C)))
    print(ACC)
    
    Csi, Precision, Recall, F1, ACC = dep.score_calculate(Model_GT, ModelPrediction2, thresholds)
    precision2, recall2, C2, ACC2 = score(Model_GT, ModelPrediction2, t)
    print("{:<20} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f}"
          .format("3. UNET_18", Csi, Precision, Recall, F1, np.mean(precision2), np.mean(recall2), np.mean(C2)))
    print(ACC)
    
    Csi, Precision, Recall, F1, ACC = dep.score_calculate(Model_GT, ModelPrediction3, thresholds)
    precision3, recall3, C3, ACC3 = score(Model_GT, ModelPrediction3, t)
    print("{:<20} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f}"
          .format("4. UNET_12", Csi, Precision, Recall, F1, np.mean(precision3), np.mean(recall3), np.mean(C3)))
    print(ACC)
    
    Csi, Precision, Recall, F1, ACC = dep.score_calculate(Model_GT, ensemble_avg, thresholds)
    Ensemble_P, Ensemble_R, Ensemble_C, Ensemble_ACC = score(Model_GT, ensemble_avg, t)
    print("{:<20} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f}"
          .format("5. HYB", Csi, Precision, Recall, F1, np.mean(Ensemble_P), np.mean(Ensemble_R), np.mean(Ensemble_C)))
    print(ACC)
    print("=" * 95)
    
    models = {"CLIM": np.expand_dims(Random_P, axis=0), "NWP": np.expand_dims(TIGG_P,axis=0), "UNET_18": np.expand_dims(precision2, axis=0),
              "UNET_12": np.expand_dims(precision3, axis=0), "HYB": np.expand_dims(Ensemble_P, axis=0)}
    interact(lambda time_step: plot_data(time_step, cbar_label='Precision', save='./Result/Precision_'+str(t)+'.png'),
             time_step=widgets.IntSlider(min=0, max=precision2.shape[0]-1, step=1, value=0));
    models = {"CLIM": np.expand_dims(Random_R, axis=0), "NWP": np.expand_dims(TIGG_R,axis=0), "UNET_18": np.expand_dims(recall2, axis=0),
              "UNET_12": np.expand_dims(recall3, axis=0), "HYB": np.expand_dims(Ensemble_R, axis=0)}
    interact(lambda time_step: plot_data(time_step, cbar_label='Recall', save='./Result/Recall_'+str(t)+'.png'),
             time_step=widgets.IntSlider(min=0, max=recall2.shape[0]-1, step=1, value=0));
    

In [ ]:
#Proportion calculation
def proportion(ground_truth, threshold):
    total_pixels = np.prod(ground_truth.shape)
    nonzero_pixels = np.sum(ground_truth > 0)
    hits = np.sum(ground_truth > threshold)
    proportion = hits / total_pixels
    conditional = hits / nonzero_pixels if nonzero_pixels > 0 else 0.0
    print(f'Proportion: {proportion}| Conditional: {conditional}')
for i in [0.0, 0.5, 10]:
    proportion(GroundTruth, i)